In [1]:
%matplotlib widget
import imageio
import numpy as np
from pathlib import Path
from omegaconf import OmegaConf
import os
from IPython.display import display, Image
os.environ["OPENCV_IO_ENABLE_OPENEXR"] = "1"
import cv2 as cv
import matplotlib.pyplot as plt
from sklearn import linear_model
import torch.nn.functional as F
import torch
import yaml

import sys
base_path = Path(os.path.abspath('')).parent
print(base_path)
sys.path.append(str(base_path))
import torch
from dataLoader import dataset_dict

def disp_im(im):
    plt.close()
    plt.cla()
    fig = plt.imshow(im)
    plt.close()
    display(fig.figure)

/home/dronelab/normalrf


In [2]:

# log_dir = Path("../log") / "tensorf"
# log_dir = Path("../log") / "fixedmip128"
# log_dir = Path("../log") / "singlebounce_samen"
exps = ["noprednorms_nl0_conserve_pb0", 'fixedmip128', 'tensorf', 'singlebounce_samen']
datadir = "/optane/nerf_datasets"
for exp in exps:
    log_dir = Path("../log") / exp
    for p in log_dir.glob("*"):
        print(p)
        pano_path = p / "imgs_test_all" / "envmaps" / "pano.exr"
        mapped_pano_path = p / "imgs_test_all" / "envmaps" / "mapped_pano.png"
        old_data_path2 = p / "imgs_test_all" / "stats_aug.yaml"
        old_data_path1 = p / "imgs_test_all" / "stats.yaml"
        old_data_path = old_data_path2 if old_data_path2.exists() else old_data_path1
        new_data_path = p / "imgs_test_all" / "stats_augn.yaml"
        config = OmegaConf.load(p / 'config.yaml')
        dname = Path(config['dataset']['scenedir']).name

        white_bg = config.dataset.white_bg if hasattr(config.dataset, 'white_bg') else True
        dataset = dataset_dict[config.dataset.dataset_name]
        test_dataset = dataset(os.path.join(datadir, config.dataset.scenedir), split='test',
                            downsample=config.dataset.downsample_train, is_stack=True, white_bg=white_bg, is_testing=True)
        norm_errs = []
        for idx in range(len(test_dataset)):
            pnorms = torch.as_tensor(imageio.imread(p / "imgs_test_all" / "world_normal" / f"{idx:03d}.png")).float()
            pnorms = (pnorms - 128) / 127
            pnorms = pnorms / torch.linalg.norm(pnorms, dim=-1, keepdim=True)
            try:
                gt_normal = test_dataset.get_normal(idx)
            except:
                break

            norm_err = torch.arccos((pnorms * gt_normal).sum(dim=-1).clip(min=1e-8, max=1-1e-8)) * 180/np.pi
            norm_err[torch.isnan(norm_err)] = 0
            norm_err *= test_dataset.acc_maps[idx].squeeze(-1)

            # f, axs = plt.subplots(2, 2)
            # axs[0, 0].imshow(gt_normal)
            # axs[1, 0].imshow(gt_norm)
            # axs[0, 1].imshow((pnorms * gt_normal).sum(dim=-1).clip(min=1e-8, max=1-1e-8))
            # axs[1, 1].imshow(norm_err)
            # plt.show()
            # break

            norm_errs.append(norm_err.mean())
        with old_data_path.open('r') as f:
            old_data = yaml.safe_load(f)
        with new_data_path.open('w') as f:
            old_data['norm_err'] = float(sum(norm_errs) / len(norm_errs)) if len(norm_errs) > 0 else 0
            yaml.dump(old_data, f)

/home/dronelab/miniconda3/envs/31/lib/python3.10/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /opt/conda/conda-bld/pytorch_1666642969563/work/aten/src/ATen/native/TensorShape.cpp:3190.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


../log/noprednorms_nl0_conserve_pb0/mic_v37_mulLdotN_no2x_elam5e-5_max0.5rough_clip_pb0_norussian_noprednorms_nl0


Loading data test (200): 100%|██████████| 200/200 [00:10<00:00, 18.55it/s]
/tmp/ipykernel_2988531/2877044086.py:25: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning dissapear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  pnorms = torch.as_tensor(imageio.imread(p / "imgs_test_all" / "world_normal" / f"{idx:03d}.png")).float()


../log/noprednorms_nl0_conserve_pb0/helmet_v37_mulLdotN_no2x_elam5e-5_max0.5rough_clip_pb0_noprednorms_nl0


Loading data test (200): 100%|██████████| 200/200 [00:12<00:00, 16.54it/s]


...

Loading data test (200): 100%|██████████| 200/200 [00:08<00:00, 24.10it/s]


../log/singlebounce_samen/ball_v38_singlebounce_samen


Loading data test (200): 100%|██████████| 200/200 [00:09<00:00, 21.07it/s]


../log/singlebounce_samen/lego_v38_singlebounce_samen


Loading data test (200): 100%|██████████| 200/200 [00:11<00:00, 18.17it/s]
